In [1]:
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append('/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/')
#import PostProcessing.panels as panels


In [1]:
pwd

'/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/PARAMETRIC/notebook'

In [3]:
cd /mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/NASTRAN

/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/NASTRAN


/mnt/c/Users/nakagawa/Desktop/python_venv/feniax/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [18]:
from pyNastran.bdf.bdf import BDF
import pandas as pd
from fem4inas.unastran.asetbuilder import BuildAsetModel
from fem4inas.unastran.aero import GenDLMPanels
import PostProcessing.panels as panels
import fem4inas.plotools.nastranvtk.bdfdef as bdfdef
import numpy as np

In [5]:
bdf = BDF(debug=None)
bdf.read_bdf("./BUG_103cao.bdf", validate=False)

bdfaero = BDF(debug=None)
bdfaero.read_bdf("./BUGaero1.bdf", validate=False, punch=False)

In [20]:
aeros = dict(RWing1=dict(nspan=2, nchord=8),
             RWing2=dict(nspan=3, nchord=8),
             RWing3=dict(nspan=9, nchord=8),
             RWing4=dict(nspan=6, nchord=8),
             RWing5=dict(nspan=4, nchord=8),
             RHTP=dict(nspan=6, nchord=8),
             LWing1=dict(nspan=2, nchord=8),
             LWing2=dict(nspan=3, nchord=8),
             LWing3=dict(nspan=9, nchord=8),
             LWing4=dict(nspan=6, nchord=8),
             LWing5=dict(nspan=4, nchord=8),
             LHTP=dict(nspan=6, nchord=8),)

aeros2ids = dict(RWing1=3504001,
                 RWing2=3500001,
                 RWing3=3501001,
                 RWing4=3502001,
                 RWing5=3503001,
                 RHTP=3600001)

for ki, i in aeros2ids.items():
    aeros[ki]['p1'] = bdfaero.caeros[i].p1
    aeros[ki]['p4'] = bdfaero.caeros[i].p4
    aeros[ki]['x12'] = bdfaero.caeros[i].x12
    aeros[ki]['x43'] = bdfaero.caeros[i].x43
    ki_l=('L'+ki[1:])
    aeros[ki_l]['p1'] = bdfaero.caeros[i].p1*np.array([1.,-1.,1.])
    aeros[ki_l]['p4'] = bdfaero.caeros[i].p4*np.array([1.,-1.,1.])
    aeros[ki_l]['x12'] = bdfaero.caeros[i].x12
    aeros[ki_l]['x43'] = bdfaero.caeros[i].x43

aeros['RWing1']['set1x'] = [1004, 2001] 
aeros['RWing2']['set1x'] = [2003, 2005, 2008, 2010] 
aeros['RWing3']['set1x'] = list(range(2012, 2030, 2))
aeros['RWing4']['set1x'] = list(range(2030, 2044, 2))
aeros['RWing5']['set1x'] = list(range(2044,2053, 2))
aeros['RHTP']['set1x'] = list(range(4000, 4014))

aeros['LWing1']['set1x'] = [1004, 10002001] 
aeros['LWing2']['set1x'] = [10002003, 10002005, 10002008, 100020010] 
aeros['LWing3']['set1x'] = list(range(10002012, 10002030, 2))
aeros['LWing4']['set1x'] = list(range(10002030, 10002044, 2))
aeros['LWing5']['set1x'] = list(range(10002044,10002053, 2))
aeros['LHTP']['set1x'] = [4000]+list(range(10004001, 10004014))

In [21]:
dlm = GenDLMPanels.from_dict(aeros) # pass your dictionary with DLM model
dlm.build_model()
dlm.model.write_bdf("./aero1.bdf") # write the bdf file

grid = panels.caero2grid(dlm.components, dlm.caero1) # build grid from dlm model
panels.build_gridmesh(grid, 'dlm_mesh_full')  #  write paraview mesh
bdfdef.vtkRef("./BUG_103cao.bdf")  # write full FE paraview

/mnt/c/Users/nakagawa/Desktop/python_venv/feniax/lib/python3.12/site-packages/pyNastran/bdf/field_writer_8.py:295: UserWarning: bad fields = ['SET1', 666007, 10002003, 10002005, 10002008, 100020010]
  warnings.warn("bad fields = %s" % fields)


---BDF Statistics---
SOL 103

bdf.spcs[1]: 1
  SPC1:    1

bdf.params: 2
  PARAM    : 2

bdf.nodes: 16358
  GRID     : 16358

bdf.coords: 2
  CORD2R   : 2

bdf.elements: 33120
  CBAR     : 66
  CBEAM    : 15908
  CQUAD4   : 17070
  CTRIA3   : 76

bdf.rigid_elements: 138
  RBE2     : 33
  RBE3     : 105

bdf.properties: 17636
  PBAR     : 46
  PBEAM    : 444
  PSHELL   : 17146

bdf.masses: 424
  CONM2    : 424

bdf.materials: 15014
  MAT1     : 6
  MAT2     : 15008

bdf.methods: 1
  EIGRL    : 1


Found 15908 CBEAM elements
Building connectivity for CBEAM
  connectivity done...
Writing CBEAM.vtu
Found 0 RBE1 elements
Found 105 RBE3 elements
Building connectivity for RBE3
  connectivity done...
Writing RBE3.vtu
Found 33 RBE2 elements
Building connectivity for RBE2
  connectivity done...
Writing RBE2.vtu
Found 0 RBAR elements
Found 0 RBAR1 elements
Found 424 CONM2 elements
Building connectivity for CONM2
  connectivity done...
Writing CONM2.vtu
Found 17070 CQUAD4 elements
Building connect